<a href="https://colab.research.google.com/github/anaperez10/AnaliticaIII-Marketing/blob/main/Avances1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>